# Handy

# References
[Nicholas Renotte's YouTube Video](https://www.youtube.com/watch?v=We1uB79Ci-w)

![Pose landmarks index](./pose_landmarks_index.png)

# Possible poses
> Sorry for the terrible drawings, were doing them in the night because I had an idea and didn't want to forget it the next morning
## 1 left
![1 left](./poses/1l.svg)
## 1 right
![1 right](./poses/1r.svg)
## 1 both
![1 both](./poses/1b.svg)

## 2 left
![2 left](./poses/2l.svg)
## 2 right
![2 right](./poses/2r.svg)
## 2 both
![2 both](./poses/2b.svg)

## 3 left
![3 left](./poses/3l.svg)
## 3 right
![3 right](./poses/3r.svg)
## 3 both
![3 both](./poses/3b.svg)

## 4 left
![4 left](./poses/4l.svg)
## 4 right
![4 right](./poses/4r.svg)
## 4 both
![4 both](./poses/4b.svg)

# Get data

In [1]:
%pip install opencv-python mediapipe numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Create CSV file if not exists
from os import path
import csv


if not path.exists("result.csv"):
    num_columns = 33

    # Create a list of column names
    column_names = ["class_name"] + [f"{coord}{i}" for i in range(1, num_columns + 1) for coord in ["x", "y", "z", "v"]]


    with open("result.csv", mode="w", newline="") as f:
        csv_writer = csv.writer(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(column_names)

In [3]:
STREAM_URL = "udp://0.0.0.0:30001" # Change it
FPS = 10 # Please first check how much time it takes to process a single frame
RESIZE_WIDTH = 960
RESIZE_HEIGHT = 540
HANDY_WINDOW = "Handy"
HANDY_MODEL_WINDOW = "Handy - model"

In [4]:
POSES = {
    '0': '0.png',
    '1': '1.png',
    '2': '2.png',
    '3': '3.png',
    '4': '4.png',
    '5': '5.png',
    '6': '6.png',
    '7': '7.png',
    '8': '8.png',
    '9': '9.png',
    '10': '10.png',
    '11': '11.png',
    '12': '12.png',
}

In [5]:
# Please change the current gathered class index
CURRENT = '0'

# Number of frames after the script will stop
FRAMES_TO_GATHER = 200 # Note that it's counted as a total sum, i.e. all frames in the directory and not only during current execution

# How many seconds to wait before starting gathering
SECONDS_TO_WAIT = 3

In [6]:
CURRENT_IMAGE = POSES[CURRENT]

In [7]:
from os import path
import time
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

def handle_frame(frame: cv2.typing.MatLike, holistic, index: int, model_frame: cv2.typing.MatLike) -> bool:
    start_time = time.time()
    # Resize frame
    frame = cv2.resize(frame, (RESIZE_WIDTH, RESIZE_HEIGHT))

    # Recolor Feed
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Make Detections
    results = holistic.process(image)

    if results.pose_landmarks is None:
        return False # It means: no detections

    # Save raw image
    cv2.imwrite(path.join("frames", f"{CURRENT}_{index}.png"), frame)

    # Pose Detections
    mp_drawing.draw_landmarks(
        model_frame,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=1, circle_radius=2),
        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=1, circle_radius=1),
    )

    # class_name,x1,y1,z1,v1,x2,y2,z2,v2,... x33,y33,z33,v33
    try:
        data = np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in results.pose_landmarks.landmark]).flatten()
        row = np.concatenate(([CURRENT], data))


        with open("result.csv", mode="a", newline="") as f:
            csv_writer = csv.writer(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(row)
    except Exception as err:
        print(err)

    cv2.imshow(HANDY_WINDOW, frame)
    cv2.imshow(HANDY_MODEL_WINDOW, model_frame)
    return True

In [8]:
from datetime import datetime, timedelta
import os
import cv2
import mediapipe as mp
import numpy as np

mp_holistic = mp.solutions.holistic

cap = cv2.VideoCapture(STREAM_URL, cv2.CAP_FFMPEG)

if not cap.isOpened():
    print("VideoCapture not opened")
    exit(-1)

ret = None

cv2.namedWindow(HANDY_MODEL_WINDOW)

while not ret:
    ret, frame = cap.read()

with mp_holistic.Holistic(
    min_detection_confidence=0.5, min_tracking_confidence=0.5
) as holistic:
    last_process = datetime.now()
    
    # Get number of already gathered frames
    # Use the os.listdir() method to get a list of all items in the folder
    if os.path.exists("frames/"):
        items = os.listdir("frames/")

        # Use a list comprehension and os.path.isfile() to filter only files
        files = [item for item in items if os.path.isfile(os.path.join("frames", item) and item.startswith(f"{CURRENT}_"))]

        # Get the number of files in the folder
        num_files = len(files)
        index = num_files
    else:
        os.mkdir("frames")
        index = 0
    
    # Do a little counting
    time_start = datetime.now() + timedelta(seconds=SECONDS_TO_WAIT)

    diff = time_start - datetime.now() 
    while diff.total_seconds() > 0:
        print("Init wait...")
        model_frame = cv2.imread(f"poses/{CURRENT_IMAGE}")
        cv2.imshow(HANDY_MODEL_WINDOW, model_frame)
        cv2.waitKey(1)
        time.sleep(0.5)
        diff = time_start - datetime.now()
    # cv2.imshow(HANDY_MODEL_WINDOW, model_frame)
    

    cv2.namedWindow(HANDY_WINDOW)
    print("INIT!")
    while True:
        ret, frame = cap.read()
        model_frame = np.zeros((560, 680, 3), dtype=np.uint8)


        if datetime.now() - last_process < timedelta(
            milliseconds = 1000 / FPS
        ):
            continue
        last_process = datetime.now()

        if not ret or frame is None:
            print("Frame empty")
            continue

        success = handle_frame(frame, holistic, index, model_frame)
        
        if success:
            print(f"Frame #{index} processed in {(datetime.now() - last_process).total_seconds()}s")
            index += 1
        else:
            print(f"Frame wasn't processed - no detection")
        if (cv2.waitKey(1) & 0xFF == ord("q")) or FRAMES_TO_GATHER <= index:
            break

cap.release()
cv2.destroyAllWindows()

Init wait...
Init wait...
Init wait...
Init wait...
Init wait...
Init wait...
INIT!
Frame wasn't processed - no detection
Frame wasn't processed - no detection
Frame wasn't processed - no detection
Frame #0 processed in 0.099349s
Frame wasn't processed - no detection
Frame #1 processed in 0.178142s
Frame wasn't processed - no detection
Frame wasn't processed - no detection
Frame wasn't processed - no detection
Frame wasn't processed - no detection
Frame wasn't processed - no detection
Frame wasn't processed - no detection
Frame wasn't processed - no detection
Frame #2 processed in 0.094294s
Frame wasn't processed - no detection
Frame wasn't processed - no detection
Frame wasn't processed - no detection
Frame #3 processed in 0.085978s
Frame wasn't processed - no detection
Frame wasn't processed - no detection
Frame wasn't processed - no detection
Frame #4 processed in 0.098208s
Frame #5 processed in 0.176765s
Frame #6 processed in 0.126482s
Frame #7 processed in 0.077494s
Frame wasn't p

In [9]:
# Get number of data per each class

all_frames = os.listdir("frames/")
result = {}

for class_name in POSES:
    # Use a list comprehension to filter files that start with "0_"
    matching_files = [filename for filename in all_frames if filename.startswith(f"{class_name}_")]

    # Get the number of matching files
    number_of_matching_files = len(matching_files)
    result[class_name] = number_of_matching_files

print(result)


{'0': 200, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0, '10': 0, '11': 0, '12': 0}
